In [15]:
import torch.nn as nn
import torch

In [16]:
multihead_attn = nn.MultiheadAttention(embed_dim=200,  # E_q(E_q必须能整除num_heads)
                                       num_heads=5,  # 注意力的数目
                                       # 默认kdim=None(即kdim=embed_dim)
                                       kdim=100,  # E_k
                                       # 默认vdim=None(即vdim=embed_dim)
                                       vdim=50,  # E_v
                                       dropout=0.1  # Dropout probability on `attn_output_weights`
                                       )

mask = torch.arange(4)[None, :] < torch.tensor([2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])[:, None]
mask = torch.unsqueeze(mask, 0)
mask = torch.repeat_interleave(mask, 40, dim=0)
mask = ~mask
print(mask.shape)
query = torch.randn(14, 8, 200)  # query.shape=(L, N, E_q);L is the target sequence length
key = torch.randn(4, 8, 100)  # key.shape=(S, N, E_k);S is the source sequence length
value = torch.randn(4, 8, 50)  # value.shape=(S, N, E_v)

torch.Size([40, 14, 4])


In [17]:
attn_output, attn_output_weights = multihead_attn(query=query,
                                                  key=key,
                                                  value=value,
                                                  # attn_mask.shape=(N * num_heads, L, S)
                                                  # a True value indicates that the corresponding position is not allowed to attend
                                                  attn_mask=mask)

In [18]:
# ★★★★★注意: pytorch输出的是头的平均注意力分数(tensorflow输出的是所有头的注意力分数)
"""
内部机制:
if need_weights:
    # average attention weights over heads
    attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
    return attn_output, attn_output_weights.sum(dim=1) / num_heads
else:
    return attn_output, None
"""
# attn_output.shape=(L, N, E_q)
attn_output.shape

torch.Size([14, 8, 200])

In [19]:
# attn_output_weights.shape=(N, L, S)
attn_output_weights.shape

torch.Size([8, 14, 4])

In [20]:
print(mask[0, :, :])
print(attn_output_weights[0, :, :])

tensor([[False, False,  True,  True],
        [False, False,  True,  True],
        [False, False,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])
tensor([[0.2539, 0.5431, 0.0000, 0.0000],
        [0.4016, 0.5902, 0.0000, 0.0000],
        [0.3744, 0.5451, 0.0000, 0.0000],
        [0.2438, 0.6758, 0.0000, 0.0000],
        [0.2059, 0.2278, 0.6774, 0.0000],
        [0.5304, 0.3810, 0.1997, 0.0000],
        [0.1313, 0.2999, 0.3711, 0.0000],
        [0.2544, 0.2238, 0.4319, 0.0000],
        [0.4367, 0.3485, 0.2122, 0.0000],
        [0.4233, 0.2540, 0.2086, 0.1455],
        [0.4418, 0.1090, 0.2379, 0.2652],
     

In [21]:
for i, j in multihead_attn.named_parameters():
    print(str(i) + ".shape=", j.shape)

q_proj_weight.shape= torch.Size([200, 200])
k_proj_weight.shape= torch.Size([200, 100])
v_proj_weight.shape= torch.Size([200, 50])
in_proj_bias.shape= torch.Size([600])
out_proj.weight.shape= torch.Size([200, 200])
out_proj.bias.shape= torch.Size([200])
